# BIGRoC: Boosting Image Generation via a Robust Classifier

--------------------------------------------------

## Comparison with other methods - CIFAR10

This colab notebook contains the needed code to experiment with our proposed algorithm for comparison with competitive metohds on CIFAR10 dataset (Section 5.3).

**How to use**

1.   Upload the notebook to colab
2.   Make sure that colab uses a GPU (Edit $\rightarrow$ Notebook settings $\rightarrow$ Hardware accelerator)

**Downloading pretrained generators:**

The pretrained image generators are stored in a [Google Drive](https://drive.google.com/drive/folders/1yN6WjMmc-pi3zHylF-I1jri7I2nZpsGJ?usp=sharing). This notebook mounts google drive and assumes that [this directory](https://drive.google.com/drive/folders/1yN6WjMmc-pi3zHylF-I1jri7I2nZpsGJ?usp=sharing) is located in your root folder in Google Drive. Therefore, please open the Google Drive's link and choose "add a shortcut to drive" and pick "My Drive" is the chosen location. By doing so, you are ready to go :)

If you rather not to mount your google drive, download the files and edit the relevant checkpoint paths accordingly.

**credits**

This notebook uses the following packages:

* [pfnet-researcrch](https://github.com/pfnet-research/sngan_projection) - For the official pretrained SN-ResNetGAN
* [robustness](https://github.com/MadryLab/robustness) - For adversarially robust classifier
* [torch fidelity](https://github.com/toshas/torch-fidelity) - For CIFAR-10 IS evaluation

# Setup & Installation

In [ ]:
!pip install torch==1.8.2+cu102 torchvision==0.9.2+cu102 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |████████████████████████████████| 804.1 MB 9.1 kB/s 
     |████████████████████████████████| 17.3 MB 27.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.8.2+cu102 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.2+cu102 which is incompatible.


In [ ]:
import torch
import numpy as np

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

torch.manual_seed(3407)
np.random.seed(3407)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/pfnet-research/sngan_projection.git
!mv sngan_projection/* ./

Cloning into 'sngan_projection'...
remote: Enumerating objects: 310, done.
remote: Total 310 (delta 0), reused 0 (delta 0), pack-reused 310
Receiving objects: 100% (310/310), 6.13 MiB | 39.72 MiB/s, done.
Resolving deltas: 100% (183/183), done.


In [ ]:
!curl https://colab.chainer.org/install | CHAINER_VERSION="==5.1.0" CUPY_VERSION="==5.1.0" sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   7559      0 --:--:-- --:--:-- --:--:--  7559
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
+ pip install -q cupy-cuda100 ==5.1.0 chainer ==5.1.0
     |████████████████████████████████| 298.6 MB 15 kB/s 
     |████████████████████████████████| 513 kB 66.6 MB/s 
+ set +ex
Installation succeeded!


In [ ]:
!pip install torch-fidelity

In [ ]:
import os, sys, time
import shutil
import numpy as np
import argparse
import chainer
from PIL import Image
from evaluation import gen_images_with_condition
import yaml
import source.yaml_utils as yaml_utils
import torch


def load_models(config):
    gen_conf = config.models['generator']
    gen = yaml_utils.load_model(gen_conf['fn'], gen_conf['name'], gen_conf['args'])
    return gen


parser = argparse.ArgumentParser()
parser.add_argument('--config_path', type=str, default='configs/sn_cifar10_conditional.yml')
parser.add_argument('--gpu', '-g', type=int, default=0)
parser.add_argument('--results_dir', type=str, default='images')
parser.add_argument('--snapshot', type=str, default='gdrive/MyDrive/ResNetGenerator_50000.npz')
parser.add_argument('--rows', type=int, default=1)
parser.add_argument('--columns', type=int, default=1)
parser.add_argument('--classes', type=int, nargs="*", default=None)
args,_ = parser.parse_known_args()
chainer.cuda.get_device_from_id(args.gpu).use()
config = yaml_utils.Config(yaml.load(open(args.config_path)))
# Model
gen = load_models(config)
gen.to_gpu(args.gpu)
out = args.results_dir
chainer.serializers.load_npz(args.snapshot, gen)
np.random.seed(1)
classes = tuple(args.classes) if args.classes is not None else np.arange(0, gen.n_classes, dtype=np.int32)

gen_imgs = []

for c in classes:
    with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
        x = gen_images_with_condition(gen, c=c, n=5000, batchsize=100)
    _, _, h, w = x.shape
    
    x = x.transpose(0, 2, 3, 1)

    for i in range(5000):
      save_path = os.path.join(out, f'{str(c)}_{i}.png')
      if not os.path.exists(out):
          os.makedirs(out)
      Image.fromarray(x[i]).save(save_path)
      gen_imgs.append(torch.tensor(x[i: i+1]))
#
gen_imgs = torch.cat(gen_imgs)

Evaluate Baseline

In [ ]:
!fidelity --gpu 0 --isc --input1 images/

Creating feature extractor "inception-v3-compat" with features ['logits_unbiased']
Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:03<00:00, 27.6MB/s]
Extracting features from input1
Looking for samples non-recursivelty in "images/" with extensions png,jpg,jpeg
Found 50000 samples
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Processing samples
Inception Score: 8.42419547380226 ± 0.10676806268476369
inception_s

In [ ]:
gen_imgs = gen_imgs / 255.
gen_imgs = gen_imgs.permute(0,3,1,2)

In [ ]:
labels = torch.ones(size=(50000,))
for c in range(10):
  labels[c*5000: (c+1) * 5000] = c
labels = labels.int()

# BIGRoC Utils

In [ ]:
!pip install robustness

In [ ]:
!wget http://andrewilyas.com/CIFAR.pt

In [ ]:
from robustness import model_utils, datasets
import torch


def create_dl_model(path, DATA='CIFAR', BATCH_SIZE=128, NUM_WORKERS=8):

    # Load dataset
    dataset_function = getattr(datasets, 'CIFAR')
    dataset = dataset_function('data')
    train_loader, test_loader = dataset.make_loaders(workers=NUM_WORKERS,
                                          batch_size=BATCH_SIZE,
                                          data_aug=False)
    # Load model
    model_kwargs = {
        'arch': 'resnet50',
        'dataset': dataset,
        'resume_path': f'./{path}'
    }
    model, _ = model_utils.make_and_restore_model(**model_kwargs)
    classifier = model.model
    classifier.eval()

    return classifier

adv_l2_model = create_dl_model(path='CIFAR.pt') 

In [ ]:
from torch import nn

# simple Module to normalize an image
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None, :, None, None]) / self.std.type_as(x)[None, :, None, None]

norm = Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])

In [ ]:
import torch
from torch import nn


class AttackerStep:
    '''
    Generic class for attacker steps, under perturbation constraints
    specified by an "origin input" and a perturbation magnitude.
    Must implement project, step, and random_perturb
    '''

    def __init__(self, orig_input, eps, step_size, use_grad=True):
        '''
        Initialize the attacker step with a given perturbation magnitude.
        Args:
            eps (float): the perturbation magnitude
            orig_input (ch.tensor): the original input
        '''
        self.orig_input = orig_input
        self.eps = eps
        self.step_size = step_size
        self.use_grad = use_grad

    def project(self, x):
        '''
        Given an input x, project it back into the feasible set
        Args:
            ch.tensor x : the input to project back into the feasible set.
        Returns:
            A `ch.tensor` that is the input projected back into
            the feasible set, that is,
        .. math:: \min_{x' \in S} \|x' - x\|_2
        '''
        raise NotImplementedError

    def step(self, x, g):
        '''
        Given a gradient, make the appropriate step according to the
        perturbation constraint (e.g. dual norm maximization for :math:`\ell_p`
        norms).
        Parameters:
            g (ch.tensor): the raw gradient
        Returns:
            The new input, a ch.tensor for the next step.
        '''
        raise NotImplementedError

    def random_perturb(self, x):
        '''
        Given a starting input, take a random step within the feasible set
        '''
        raise NotImplementedError

    def to_image(self, x):
        '''
        Given an input (which may be in an alternative parameterization),
        convert it to a valid image (this is implemented as the identity
        function by default as most of the time we use the pixel
        parameterization, but for alternative parameterizations this functino
        must be overriden).
        '''
        return x


# L2 threat model
class L2Step(AttackerStep):
    """
    Attack step for :math:`\ell_\infty` threat model. Given :math:`x_0`
    and :math:`\epsilon`, the constraint set is given by:
    .. math:: S = \{x | \|x - x_0\|_2 \leq \epsilon\}
    """

    def project(self, x):
        """
        """
        if self.orig_input is None: self.orig_input = x.detach()
        self.orig_input = self.orig_input.cuda()
        diff = x - self.orig_input
        diff = diff.renorm(p=2, dim=0, maxnorm=self.eps)
        return torch.clamp(self.orig_input + diff, 0, 1)

    def step(self, x, g):
        """
        """
        l = len(x.shape) - 1
        g_norm = torch.norm(g.reshape(g.shape[0], -1), dim=1).view(-1, *([1] * l))
        scaled_g = g / (g_norm + 1e-10)
        return x + scaled_g * self.step_size


def targeted_pgd_l2(model, X, y, num_iter, eps=30, step_size=0.5):
    steper = L2Step(eps=eps, orig_input=None, step_size=step_size)
    for t in range(num_iter):
        X = X.clone().detach().requires_grad_(True).cuda()
        loss = nn.CrossEntropyLoss(reduction='none')(model(norm(X)), y)
        loss = torch.mean(loss)
        grad, = torch.autograd.grad(-1 * loss, [X])
        X = steper.step(X, grad)
        X = steper.project(X)
    return X.detach()

# BIGRoC Application

In [ ]:
from tqdm.notebook import tqdm


eps = 1.8
num_steps = 7
step_size = 1.5 * eps / num_steps


boosted_imgs = []
b_size = 100
num_b = 50000 // b_size
for i in tqdm(range(num_b)):
  x, y = gen_imgs[i * b_size: (i + 1) * b_size].cuda(), labels[i * b_size: (i + 1) * b_size].cuda()
  gen_imgs_boosted = targeted_pgd_l2(model=adv_l2_model, X=x.data, y=y.long(), num_iter=num_steps, eps=eps, step_size=step_size)
  boosted_imgs.append(gen_imgs_boosted)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
import matplotlib.pyplot as plt

idx = 7

plt.figure()
plt.imshow(boosted_imgs[0][idx].permute(1,2,0).detach().cpu().numpy())

plt.figure()
plt.imshow(gen_imgs[idx].permute(1,2,0).detach().cpu().numpy())

plt.figure()
plt.imshow((boosted_imgs[0][idx] - gen_imgs[idx].cuda()).permute(1,2,0).detach().cpu().numpy())

In [ ]:
X_boost = torch.cat(boosted_imgs)
X_boost = X_boost[:(num_b * b_size)]
X_boost = (X_boost * 255).int()
X_boost = X_boost.detach().cpu().permute(0,2,3,1).numpy()
X_boost = X_boost.astype(np.uint8)

In [ ]:
for i, img in tqdm(enumerate(X_boost)):
  save_path = os.path.join(f'boosted_images_eps_{eps}', f'{i}.png')
  if not os.path.exists(f'boosted_images_eps_{eps}'):
      os.makedirs(f'boosted_images_eps_{eps}')
  Image.fromarray(img).save(save_path)

0it [00:00, ?it/s]

In [ ]:
!fidelity --gpu 0 --isc --input1 boosted_images_eps_1.8/

Creating feature extractor "inception-v3-compat" with features ['logits_unbiased']
Extracting features from input1
Looking for samples non-recursivelty in "boosted_images_eps_1.8/" with extensions png,jpg,jpeg
Found 50000 samples
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Processing samples
Inception Score: 9.333590194871679 ± 0.08018190897178912
inception_score_mean: 9.33359
inception_score_std: 0.08018191
